In [1]:
import pandas as pd
import numpy as np

In [2]:
data_arr = pd.read_parquet("data\\arr_data.parquet")
data_arr

,yr,mon,day_of_week,dep_hour,arr_hour,u_carrier,origin_airport_code,dest_airport_code,arr_delay_new,arr_del15,distance_grp
0,2003,1,3,9,11,US,PIT,LAX,0,0,9
1,2003,1,3,9,12,AA,JFK,LAX,0,0,10
2,2003,1,3,8,10,B6,BUF,JFK,0,0,2
3,2003,1,3,9,12,AS,DCA,SEA,2,0,10
4,2003,1,3,8,12,HP,PHX,ORD,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...
68272049,2012,12,1,11,12,OO,MSP,IMT,0,0,2
68272050,2012,12,1,19,20,OO,MSP,IMT,0,0,2
68272051,2012,12,1,11,12,OO,MSP,INL,0,0,2
68272052,2012,12,1,11,12,OO,SLC,PIH,7,0,1


In [3]:
data_dep = pd.read_parquet("data\\dep_data.parquet")
data_dep

,yr,mon,day_of_week,dep_hour,arr_hour,u_carrier,origin_airport_code,dest_airport_code,dep_delay_new,dep_delay15,distance_grp
0,2003,1,3,9,11,US,PIT,LAX,0,0,9
1,2003,1,3,9,12,AA,JFK,LAX,0,0,10
2,2003,1,3,8,10,B6,BUF,JFK,1,0,2
3,2003,1,3,9,12,AS,DCA,SEA,0,0,10
4,2003,1,3,8,12,HP,PHX,ORD,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...
68272049,2012,12,1,11,12,OO,MSP,IMT,0,0,2
68272050,2012,12,1,19,20,OO,MSP,IMT,0,0,2
68272051,2012,12,1,11,12,OO,MSP,INL,0,0,2
68272052,2012,12,1,11,12,OO,SLC,PIH,0,0,1


In [1]:
data_arr.day_of_week.value_counts()

NameError: name 'data_arr' is not defined

In [16]:
data_arr.distance_grp.value_counts()

2     15088451
3     14534261
4     10413777
1      9176522
5      6172877
7      3603426
6      3269883
8      1842577
10     1632444
9      1376625
11     1161211
Name: distance_grp, dtype: int64

In [33]:
data_arr.u_carrier.value_counts().count()

24

In [29]:
data_arr.origin_airport_code.value_counts()

ATL    5055909
ORD    4452054
DFW    3765836
LAX    3157224
DEN    2981342
        ...   
SHD         18
PIR          8
OGD          5
CDV          3
PVU          1
Name: origin_airport_code, Length: 331, dtype: int64

In [30]:
data_arr.dest_airport_code.value_counts()

ATL    5035476
ORD    4448764
DFW    3754987
LAX    3156084
DEN    2977174
        ...   
AZA         47
IMT         30
SHD         18
PIR          8
CBM          1
Name: dest_airport_code, Length: 329, dtype: int64

# origin and dest

In [36]:
# get pairwise count of each origin and dest
pairwise_count = data_arr.groupby(['origin_airport_code', 'dest_airport_code']).count()
pairwise_count

yr  mon  day_of_week  dep_hour  \
origin_airport_code dest_airport_code                                   
ABE                 ABE                 0    0            0         0   
                    ABI                 0    0            0         0   
                    ABQ                 0    0            0         0   
                    ABR                 0    0            0         0   
                    ABY                 0    0            0         0   
...                                    ..  ...          ...       ...   
YUM                 VPS                 0    0            0         0   
                    WYS                 0    0            0         0   
                    XNA                 0    0            0         0   
                    YKM                 0    0            0         0   
                    YUM                 0    0            0         0   

                                       arr_hour  u_carrier  arr_delay_new  \
origin_airport_code dest_airport_code                                       
ABE                 ABE                       0          0              0   
                    ABI                       0          0              0   
                    ABQ                       0          0              0   
                    ABR                       0          0              0   
                    ABY                       0          0              0   
...                                         ...        ...            ...   
YUM                 VPS                       0          0              0   
                    WYS                       0          0              0   
                    XNA                       0          0              0   
                    YKM                       0          0              0   
                    YUM                       0          0              0   

                                       arr_del15  distance_grp  
origin_airport_code dest_airport_code                           
ABE                 ABE                        0             0  
                    ABI                        0             0  
                    ABQ                        0             0  
                    ABR                        0             0  
                    ABY                        0             0  
...                                          ...           ...  
YUM                 VPS                        0             0  
                    WYS                        0             0  
                    XNA                        0             0  
                    YKM                        0             0  
                    YUM                        0             0  

[108899 rows x 9 columns]

In [38]:
# removing zero rows
pairwise_count = pairwise_count[pairwise_count.yr > 0]

In [46]:
airport_pairs = pairwise_count.reset_index()[["origin_airport_code", "dest_airport_code"]]
airport_pairs

,origin_airport_code,dest_airport_code
0,ABE,ATL
1,ABE,BWI
2,ABE,CLT
3,ABE,DTW
4,ABE,JFK
...,...,...
4277,YKM,SLC
4278,YUM,LAS
4279,YUM,LAX
4280,YUM,PHX


In [60]:
sorted(airport_pairs.dest_airport_code.unique().tolist())

['ABE',
 'ABI',
 'ABQ',
 'ABR',
 'ABY',
 'ACK',
 'ACT',
 'ACV',
 'ACY',
 'AEX',
 'AGS',
 'ALB',
 'ALO',
 'AMA',
 'ANC',
 'APF',
 'APN',
 'ART',
 'ASE',
 'ATL',
 'ATW',
 'AUS',
 'AVL',
 'AVP',
 'AZA',
 'AZO',
 'BDL',
 'BFL',
 'BGM',
 'BGR',
 'BHM',
 'BIL',
 'BIS',
 'BJI',
 'BKG',
 'BLI',
 'BMI',
 'BNA',
 'BOI',
 'BOS',
 'BPT',
 'BQK',
 'BQN',
 'BRD',
 'BRO',
 'BTM',
 'BTR',
 'BTV',
 'BUF',
 'BUR',
 'BWI',
 'BZN',
 'CAE',
 'CAK',
 'CBM',
 'CDC',
 'CEC',
 'CHA',
 'CHO',
 'CHS',
 'CIC',
 'CID',
 'CLD',
 'CLE',
 'CLL',
 'CLT',
 'CMH',
 'CMI',
 'CMX',
 'COD',
 'COS',
 'COU',
 'CPR',
 'CRP',
 'CRW',
 'CSG',
 'CVG',
 'CWA',
 'CYS',
 'DAB',
 'DAL',
 'DAY',
 'DBQ',
 'DCA',
 'DEN',
 'DFW',
 'DHN',
 'DLH',
 'DRO',
 'DRT',
 'DSM',
 'DTW',
 'EAU',
 'ECP',
 'EFD',
 'EGE',
 'EKO',
 'ELM',
 'ELP',
 'ERI',
 'ESC',
 'EUG',
 'EVV',
 'EWN',
 'EWR',
 'EYW',
 'FAI',
 'FAR',
 'FAT',
 'FAY',
 'FCA',
 'FLG',
 'FLL',
 'FLO',
 'FNT',
 'FSD',
 'FSM',
 'FWA',
 'GCC',
 'GCK',
 'GEG',
 'GFK',
 'GGG',
 'GJT',
 'GNV',


In [55]:
airport_pairs[airport_pairs.origin_airport_code == "ATL"][
            "dest_airport_code"
        ].tolist()

['ABE',
 'ABQ',
 'ABY',
 'ACY',
 'AEX',
 'AGS',
 'ALB',
 'ANC',
 'APF',
 'ASE',
 'ATW',
 'AUS',
 'AVL',
 'AVP',
 'AZO',
 'BDL',
 'BGM',
 'BGR',
 'BHM',
 'BIL',
 'BKG',
 'BMI',
 'BNA',
 'BOI',
 'BOS',
 'BPT',
 'BQK',
 'BQN',
 'BTR',
 'BTV',
 'BUF',
 'BUR',
 'BWI',
 'BZN',
 'CAE',
 'CAK',
 'CBM',
 'CHA',
 'CHO',
 'CHS',
 'CID',
 'CLE',
 'CLT',
 'CMH',
 'CMI',
 'COS',
 'COU',
 'CRP',
 'CRW',
 'CSG',
 'CVG',
 'DAB',
 'DAL',
 'DAY',
 'DCA',
 'DEN',
 'DFW',
 'DHN',
 'DSM',
 'DTW',
 'ECP',
 'EGE',
 'ELP',
 'ERI',
 'EVV',
 'EWN',
 'EWR',
 'EYW',
 'FAY',
 'FCA',
 'FLL',
 'FLO',
 'FNT',
 'FSD',
 'FSM',
 'FWA',
 'GNV',
 'GPT',
 'GRB',
 'GRK',
 'GRR',
 'GSO',
 'GSP',
 'GTR',
 'GUC',
 'HDN',
 'HHH',
 'HKY',
 'HNL',
 'HOU',
 'HPN',
 'HSV',
 'HTS',
 'IAD',
 'IAH',
 'ICT',
 'ILG',
 'ILM',
 'IND',
 'ISO',
 'ISP',
 'JAC',
 'JAN',
 'JAX',
 'JFK',
 'LAN',
 'LAS',
 'LAW',
 'LAX',
 'LEX',
 'LFT',
 'LGA',
 'LGB',
 'LIT',
 'LNK',
 'LWB',
 'LYH',
 'MCI',
 'MCN',
 'MCO',
 'MDT',
 'MDW',
 'MEI',
 'MEM',
 'MFE',


# distance grp

In [63]:
# sanity check
pairwise_dist_groups_arr = data_arr.groupby(['origin_airport_code', 'dest_airport_code']).distance_grp.nunique()
pairwise_dist_groups_arr = pairwise_dist_groups_arr[pairwise_dist_groups_arr > 0]
pairwise_dist_groups_arr.value_counts()

origin_airport_code  dest_airport_code
ABE                  ATL                  1
                     BWI                  1
                     CLT                  1
                     DTW                  1
                     JFK                  1
                                         ..
YKM                  SLC                  1
YUM                  LAS                  1
                     LAX                  1
                     PHX                  1
                     SLC                  1
Name: distance_grp, Length: 4282, dtype: int64

In [8]:
airport_pairs_dist = pd.concat([data_arr[['origin_airport_code', 'dest_airport_code', 'distance_grp']],
                               data_dep[['origin_airport_code', 'dest_airport_code', 'distance_grp']]]
                              ).drop_duplicates()

In [10]:
airport_pairs_dist

,origin_airport_code,dest_airport_code,distance_grp
0,PIT,LAX,9
1,JFK,LAX,10
2,BUF,JFK,2
3,DCA,SEA,10
4,PHX,ORD,6
...,...,...,...
67918746,MCO,SHD,3
68137347,MSP,IMT,2
68190452,JFK,PSP,10
68194143,LGA,EYW,5


In [9]:
# sanity check
airport_pairs_dist.groupby(['origin_airport_code', 'dest_airport_code']).distance_grp.nunique().value_counts()

0    104617
1      4282
Name: distance_grp, dtype: int64

In [12]:
airport_pairs_dist.to_csv("data/airport_pairs.csv", index = False)

In [13]:
airport_pairs = pd.read_csv("data/airport_pairs.csv")
airport_pairs

,origin_airport_code,dest_airport_code,distance_grp
0,PIT,LAX,9
1,JFK,LAX,10
2,BUF,JFK,2
3,DCA,SEA,10
4,PHX,ORD,6
...,...,...,...
4277,MCO,SHD,3
4278,MSP,IMT,2
4279,JFK,PSP,10
4280,LGA,EYW,5


In [79]:
airport_pairs[(airport_pairs.origin_airport_code == "YUM") &
    (airport_pairs.dest_airport_code == "SLC")]['distance_grp'].values[0]

3

# carriers

In [14]:
# get carriers
print(sorted(data_arr.u_carrier.unique()))
print(sorted(data_dep.u_carrier.unique()))

['9E', 'AA', 'AS', 'B6', 'CO', 'DH', 'DL', 'EV', 'F9', 'FL', 'HA', 'HP', 'KH', 'MQ', 'NW', 'OH', 'OO', 'TZ', 'UA', 'US', 'VX', 'WN', 'XE', 'YV']
['9E', 'AA', 'AS', 'B6', 'CO', 'DH', 'DL', 'EV', 'F9', 'FL', 'HA', 'HP', 'KH', 'MQ', 'NW', 'OH', 'OO', 'TZ', 'UA', 'US', 'VX', 'WN', 'XE', 'YV']


# make datasets for line plot with same orig and dest

In [21]:
orig_dest_grp_arr = data_arr.groupby(["yr","mon", "origin_airport_code",  "dest_airport_code"]).agg(
    max = ('arr_delay_new', max),  
    min = ('arr_delay_new', min),  
    mean = ('arr_delay_new', np.mean),
#    count = ('arr_delay_new', count)
)
orig_dest_grp_arr

max  min  mean
yr   mon origin_airport_code dest_airport_code                
2003 1   ABE                 ABE                NaN  NaN   NaN
                             ABI                NaN  NaN   NaN
                             ABQ                NaN  NaN   NaN
                             ABR                NaN  NaN   NaN
                             ABY                NaN  NaN   NaN
...                                             ...  ...   ...
2012 12  YUM                 VPS                NaN  NaN   NaN
                             WYS                NaN  NaN   NaN
                             XNA                NaN  NaN   NaN
                             YKM                NaN  NaN   NaN
                             YUM                NaN  NaN   NaN

[13067880 rows x 3 columns]

In [19]:
orig_dest_grp_arr[~orig_dest_grp_arr['max'].isna()].reset_index().to_csv("data/grp_orig_dest_arr.csv", index = False)

In [36]:
orig_dest_grp_dep = data_dep.groupby(["yr","mon", "origin_airport_code",  "dest_airport_code"]).agg(
    max = ('dep_delay_new', max),  
    min = ('dep_delay_new', min),  
    mean = ('dep_delay_new', np.mean),
#    count = ('dep_delay_new', count)
)
orig_dest_grp_dep

max  min  mean
yr   mon origin_airport_code dest_airport_code                
2003 1   ABE                 ABE                NaN  NaN   NaN
                             ABI                NaN  NaN   NaN
                             ABQ                NaN  NaN   NaN
                             ABR                NaN  NaN   NaN
                             ABY                NaN  NaN   NaN
...                                             ...  ...   ...
2012 12  YUM                 VPS                NaN  NaN   NaN
                             WYS                NaN  NaN   NaN
                             XNA                NaN  NaN   NaN
                             YKM                NaN  NaN   NaN
                             YUM                NaN  NaN   NaN

[13067880 rows x 3 columns]

In [37]:
orig_dest_grp_dep[~orig_dest_grp_dep['max'].isna()].reset_index()

,yr,mon,origin_airport_code,dest_airport_code,max,min,mean
0,2003,1,ABE,ATL,68.0,0.0,7.965517
1,2003,1,ABE,CLT,101.0,0.0,5.066667
2,2003,1,ABE,DTW,49.0,0.0,6.225806
3,2003,1,ABE,ORD,208.0,0.0,20.112360
4,2003,1,ABI,DFW,102.0,0.0,4.485149
...,...,...,...,...,...,...,...
356571,2012,12,XNA,IAH,366.0,0.0,23.322222
356572,2012,12,XNA,LGA,550.0,0.0,17.482143
356573,2012,12,XNA,ORD,924.0,0.0,31.179592
356574,2012,12,YUM,LAX,310.0,0.0,18.141176


In [38]:
orig_dest_grp_dep[~orig_dest_grp_dep['max'].isna()].reset_index().to_csv("data/grp_orig_dest_dep.csv", index = False)

In [4]:
import plotly.express as px

orig_dest_arr_df = pd.read_csv("data/grp_orig_dest_arr.csv")
orig_dest_dep_df = pd.read_csv("data/grp_orig_dest_dep.csv")

In [16]:
def update_plot(orig, dest):
    
    orig_dest_plot = px.line()
    if all([orig, dest]):
        dep_subset = orig_dest_dep_df[(orig_dest_dep_df.origin_airport_code == orig) &
        (orig_dest_dep_df.dest_airport_code == dest)]
        dep_subset['yr_mon'] = dep_subset.yr.astype(str) + "-" + dep_subset.mon.map("{:02}".format)
        dep_subset['type'] = "departure"
        
        arr_subset = orig_dest_arr_df[(orig_dest_arr_df.origin_airport_code == orig) &
        (orig_dest_arr_df.dest_airport_code == dest)]
        arr_subset['yr_mon'] = arr_subset.yr.astype(str) + "-" + arr_subset.mon.map("{:02}".format)
        arr_subset['type'] = "arrival"
        
        df = pd.concat([dep_subset[["yr_mon", "mean", "type"]], 
                        arr_subset[["yr_mon", "mean", "type"]]])
        orig_dest_plot = px.line(df, x = "yr_mon", y = "mean", color = "type", markers = True)
    return orig_dest_plot
update_plot("ATL", "ABQ")

c:\users\jiatong\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\users\jiatong\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\users\jiatong\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

# carrier

In [17]:
carrier_grp_arr = data_arr.groupby(["yr","mon", "u_carrier"]).agg(
    max = ('arr_delay_new', max),  
    min = ('arr_delay_new', min),  
    mean = ('arr_delay_new', np.mean),
#    count = ('arr_delay_new', count)
)
carrier_grp_arr

max  min       mean
yr   mon u_carrier                        
2003 1   9E            NaN  NaN        NaN
         AA         1098.0  0.0   7.827011
         AS          300.0  0.0   8.969894
         B6          785.0  0.0   8.803249
         CO          392.0  0.0   7.530320
...                    ...  ...        ...
2012 12  US          528.0  0.0   8.513824
         VX          635.0  0.0  16.157929
         WN          562.0  0.0  14.651770
         XE            NaN  NaN        NaN
         YV          414.0  0.0   7.743221

[2880 rows x 3 columns]

In [20]:
carrier_grp_arr[~carrier_grp_arr['max'].isna()].reset_index().to_csv("data/grp_carrier_arr.csv", index = False)

In [22]:
carrier_grp_dep = data_dep.groupby(["yr","mon", "u_carrier"]).agg(
    max = ('dep_delay_new', max),  
    min = ('dep_delay_new', min),  
    mean = ('dep_delay_new', np.mean),
#    count = ('dep_delay_new', count)
)
carrier_grp_dep

max  min       mean
yr   mon u_carrier                        
2003 1   9E            NaN  NaN        NaN
         AA         1122.0  0.0   6.461317
         AS          313.0  0.0   8.710268
         B6          743.0  0.0   9.105295
         CO          400.0  0.0   4.555950
...                    ...  ...        ...
2012 12  US          558.0  0.0   6.989176
         VX          619.0  0.0  15.770661
         WN          560.0  0.0  16.637198
         XE            NaN  NaN        NaN
         YV          403.0  0.0   7.208124

[2880 rows x 3 columns]

In [23]:
carrier_grp_dep[~carrier_grp_dep['max'].isna()].reset_index().to_csv("data/grp_carrier_dep.csv", index = False)

# departure hour

In [25]:
deph_grp_arr = data_arr.groupby(["yr","mon", "dep_hour"]).agg(
    max = ('arr_delay_new', max),  
    min = ('arr_delay_new', min),  
    mean = ('arr_delay_new', np.mean),
#    count = ('arr_delay_new', count)
)
deph_grp_arr

max  min       mean
yr   mon dep_hour                      
2003 1   0          785    0   5.534602
         1          132    0   4.375358
         2          103    0   8.045455
         5          658    0   5.684211
         6          988    0   5.788395
...                 ...  ...        ...
2012 12  19         892    0  17.218335
         20         864    0  18.131505
         21         831    0  18.723816
         22         726    0  12.582921
         23        1080    0  12.487664

[2802 rows x 3 columns]

In [26]:
deph_grp_arr[~deph_grp_arr['max'].isna()].reset_index().to_csv("data/grp_deph_arr.csv", index = False)

In [27]:
deph_grp_dep = data_dep.groupby(["yr","mon", "dep_hour"]).agg(
    max = ('dep_delay_new', max),  
    min = ('dep_delay_new', min),  
    mean = ('dep_delay_new', np.mean),
#    count = ('dep_delay_new', count)
)
deph_grp_dep

max  min       mean
yr   mon dep_hour                      
2003 1   0          743    0   4.942042
         1          126    0   2.687679
         2          111    0   4.454545
         5          638    0   4.018421
         6          987    0   3.729834
...                 ...  ...        ...
2012 12  19         875    0  17.963018
         20         845    0  18.377646
         21         841    0  19.455513
         22         703    0  13.871078
         23        1088    0  15.585564

[2802 rows x 3 columns]

In [28]:
deph_grp_dep[~deph_grp_dep['max'].isna()].reset_index().to_csv("data/grp_deph_dep.csv", index = False)

# arrival hour

In [29]:
arrh_grp_arr = data_arr.groupby(["yr","mon", "arr_hour"]).agg(
    max = ('arr_delay_new', max),  
    min = ('arr_delay_new', min),  
    mean = ('arr_delay_new', np.mean),
#    count = ('arr_delay_new', count)
)
arrh_grp_arr

max  min       mean
yr   mon arr_hour                      
2003 1   0         1435    0  31.232260
         1          134    0   7.949541
         2          196    0  11.121951
         4           78    0   4.541063
         5          806    0   4.662215
...                 ...  ...        ...
2012 12  19        1175    0  16.920462
         20         856    0  17.245380
         21        1159    0  17.507102
         22         864    0  17.637816
         23         780    0  16.884765

[2871 rows x 3 columns]

In [30]:
arrh_grp_arr[~arrh_grp_arr['max'].isna()].reset_index().to_csv("data/grp_arrh_arr.csv", index = False)

In [31]:
arrh_grp_dep = data_dep.groupby(["yr","mon", "arr_hour"]).agg(
    max = ('dep_delay_new', max),  
    min = ('dep_delay_new', min),  
    mean = ('dep_delay_new', np.mean),
#    count = ('dep_delay_new', count)
)
arrh_grp_dep

max  min       mean
yr   mon arr_hour                      
2003 1   0          370    0   9.011783
         1          145    0   8.590214
         2          136    0   8.170732
         4           85    0   3.217391
         5          817    0   3.582359
...                 ...  ...        ...
2012 12  19        1183    0  17.242812
         20         801    0  17.591636
         21        1188    0  18.168526
         22         845    0  18.432837
         23         786    0  18.234622

[2871 rows x 3 columns]

In [32]:
arrh_grp_dep[~arrh_grp_dep['max'].isna()].reset_index().to_csv("data/grp_arrh_dep.csv", index = False)

# making options dictionary

In [46]:
# writing options to dictionary
import json
options_dict = {"dayofweek": [{'label':'Mon', 'value':1}, {'label':'Tue','value':2}, {'label':'Wed','value':3},
                 {'label':'Thu','value':4}, {'label':'Fri','value':5}, {'label':'Sat','value':6}, {'label':'Sun','value':7}],
                "year": [2015, 2014, 2013], #2022, 2021, 2020, 2019, 2018, 2017, 2016, 
               "month": [
                {'label':'Jan', 'value':1},
                {'label':'Feb','value':2},
                {'label':'Mar','value':3},
                {'label':'Apr','value':4},
                {'label':'May','value':5},
                {'label':'Jun','value':6},
                {'label':'Jul','value':7},
                {'label':'Aug','value':8},
                {'label':'Sep','value':9},
                {'label':'Oct','value':10},
                {'label':'Nov','value':11},
                {'label':'Dec','value':12},                
            ],
               "carrier": [
                {'label':'Endeavor Air', 'value':'9E'},
                {'label':'American Airlines','value':'AA'},
                {'label':'Alaska Airlines','value':'AS'},
                {'label':'JetBlue','value':'B6'},
                {'label':'Continental Airlines','value':'CO'},
                {'label':'Discovery Airways','value':'DH'},
                {'label':'Delta Air Lines','value':'DL'},
                {'label':'aha!','value':'EV'},
                {'label':'Frontier Airlines','value':'F9'},
                {'label':'AirTran Airways','value':'FL'},
                {'label':'Hawaiian Airlines','value':'HA'},
                {'label':'America West Airlines','value':'HP'},                     
                {'label':'Aloha Air Cargo', 'value':'KH'},
                {'label':'Envoy Air','value':'MQ'},
                {'label':'Northwest Airlines','value':'NW'},
                {'label':'PSA Airlines','value':'OH'},
                {'label':'SkyWest Airlines','value':'OO'},
                {'label':'ATA Airlines','value':'TZ'},
                {'label':'United Airlines','value':'UA'},
                {'label':'US Airways','value':'US'},
                {'label':'Virgin America','value':'VX'},
                {'label':'Southwest Airlines','value':'WN'},
                {'label':'JSX','value':'XE'},
                {'label':'Mesa Airlines','value':'YV'}]}                
                   
                   

with open("data/options_dict.txt","w") as file:
  file.write(json.dumps(options_dict))

In [47]:
# check dictionary content
with open("data/options_dict.txt", "r") as file:
 file_content = file.read()
eval(file_content)

{'dayofweek': [{'label': 'Mon', 'value': 1},
  {'label': 'Tue', 'value': 2},
  {'label': 'Wed', 'value': 3},
  {'label': 'Thu', 'value': 4},
  {'label': 'Fri', 'value': 5},
  {'label': 'Sat', 'value': 6},
  {'label': 'Sun', 'value': 7}],
 'year': [2015, 2014, 2013],
 'month': [{'label': 'Jan', 'value': 1},
  {'label': 'Feb', 'value': 2},
  {'label': 'Mar', 'value': 3},
  {'label': 'Apr', 'value': 4},
  {'label': 'May', 'value': 5},
  {'label': 'Jun', 'value': 6},
  {'label': 'Jul', 'value': 7},
  {'label': 'Aug', 'value': 8},
  {'label': 'Sep', 'value': 9},
  {'label': 'Oct', 'value': 10},
  {'label': 'Nov', 'value': 11},
  {'label': 'Dec', 'value': 12}],
 'carrier': [{'label': 'Endeavor Air', 'value': '9E'},
  {'label': 'American Airlines', 'value': 'AA'},
  {'label': 'Alaska Airlines', 'value': 'AS'},
  {'label': 'JetBlue', 'value': 'B6'},
  {'label': 'Continental Airlines', 'value': 'CO'},
  {'label': 'Discovery Airways', 'value': 'DH'},
  {'label': 'Delta Air Lines', 'value': 'DL'}

In [53]:
ls1=[dic['label'] for dic in options_dict['carrier']]

ls2=[dic['value'] for dic in options_dict['carrier']]

with open("data/carrier_dict.txt","w") as file:
  file.write(json.dumps(dict(zip(ls2,ls1))))